<a href="https://colab.research.google.com/github/Sripathm2/UCLA_CS_245_Project5/blob/main/GNN/County_Data_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/CS245

/content/drive/MyDrive/CS245


In [ ]:
import pandas as pd
import numpy as np

In [18]:
sci_path = '/content/drive/MyDrive/CS245/county_county_aug2020.tsv'
covid_path = '/content/drive/MyDrive/CS245/Clean.csv'

In [14]:
def load_file(SCI_data, covid_data):
  sci = pd.read_csv(SCI_data, sep='\t').values
  covid = pd.read_csv(covid_data, sep=',').values
  fips = sci[:3229][:,1].tolist()

  present_list = np.zeros(3229)
  for i in range(0,covid.shape[0],67):
    covid_fip = covid[i,1]
    for f in range(len(fips)):
      if covid_fip == fips[f]:
        present_list[f] = 1

  absent_list = []
  for i in range(3229):
    if present_list[i] == 0:
      absent_list.append(fips[i])

  sci = pd.read_csv(SCI_data, sep='\t', na_values=absent_list).values
  return sci, covid, fips

In [16]:
def build_adj_mat(sci):
  adj = np.zeros((3215,3215))
  for i in range(3215):
    end_idx = (i+1)*3215
    start_idx = i*3215
    sci_i = sci[start_idx:end_idx]
    adj[i] = sci_i[:,2]
  return adj

In [54]:
def build_county_data(covid, window=6):
  max_seq = int(covid.shape[0] / 3215)
  timestep = int(max_seq - window + 1)
  
  X = np.zeros([timestep,3215,window-1])
  y = np.zeros([timestep,3215,1])

  for i in range(3215):
    for j in range(timestep):
      X[j,i] = covid[i*max_seq+j:i*max_seq+j+window-1,2]
      y[j,i] = covid[i*max_seq+j+window-2,2]
  
  return X, y


In [56]:
sci, covid, fips = load_file(sci_path, covid_path)
adj = build_adj_mat(sci)
X, y = build_county_data(covid)